In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import re 
import time
from pandas.io.json import json_normalize

In [2]:
challengergame_df = pd.read_csv('../Data/Match_Data/challenger_match.csv')
challengergame_df2 = pd.read_csv('../Data/Match_Data/challenger_match_V2.csv')

ingame_df = pd.read_pickle('../Data/Match_Data/match_data_version2.pickle')

In [3]:
chall_game = pd.concat([challengergame_df,challengergame_df2],axis = 0)
chall_game_unique = chall_game.drop_duplicates('gameId')
ingame_df['gameId'] = ingame_df['gameId'].astype('int')

In [4]:
chall_match = pd.merge(ingame_df,chall_game_unique[['gameId','accountId']], on = 'gameId', how = 'left')\
                                    .drop(columns=['status.message','status.status_code'])\
                                    .dropna(axis=0)\
                                    .reset_index(drop=True)

In [5]:
match_df = chall_match[chall_match["queueId"]==420].copy().reset_index(drop=True)

In [6]:
champs = pd.read_csv("../Data/Champions_Items_Info/champions.csv")

In [7]:
def preparing_dataframe(df, lower:int, upper:int):
    df_storage = pd.DataFrame()
    for i in range(lower,upper):
        
        ''' Getting macro information on the game: 
            This is the ID of the game and the duration of the game.
        '''
        
        gameId = df["gameId"].iloc[i]
        gameDuration = df["gameDuration"].iloc[i]
        
        ''' 
        Now looking into the teams statistics, where we want to get information on 
        first blood, first tower, first dragon. If this is True for team "Blue", 
        the value will be set to 1, otherwise it will be set to 0
        '''
        
        blueWins = np.where(df['teams'].iloc[i][0]['win']=="Win", 1, 0)
        firstblood_blue = np.where(df['teams'].iloc[i][0]['firstBlood']==True,1,0)
        firstTower_blue = np.where(df['teams'].iloc[i][0]['firstTower']==True,1,0)
        firstDragon_blue = np.where(df['teams'].iloc[i][0]['firstDragon']==True,1,0)       
        
        '''
        Now looking into the different participants
        In there we want to get information on average gold/minute and exp/minute for the early game
        '''
        champion_list_blue = []
        champion_list_red = []
        
        gold_per_minute_blue = 0
        xp_per_minute_blue = 0
        cs_per_minute_blue = 0
        damage_taken_per_minute_blue = 0

        gold_per_minute_red = 0
        xp_per_minute_red = 0
        cs_per_minute_red = 0
        damage_taken_per_minute_red = 0
        
        for j in range(len(df["participants"].iloc[i])):
            if df["participants"].iloc[i][j]["teamId"]==100:
                champion_list_blue.append(df["participants"].iloc[i][j]["championId"])
                try:
                    gpm = df["participants"].iloc[i][j]["timeline"]["goldPerMinDeltas"]["0-10"]
                except:
                    gpm = 0
                try:
                    xpm = df["participants"].iloc[i][j]["timeline"]["xpPerMinDeltas"]["0-10"]
                except:
                    xpm = 0
                try:
                    cspm = df["participants"].iloc[i][j]["timeline"]["creepsPerMinDeltas"]["0-10"]
                except:
                    cspm = 0
                try:
                    dtpm = df["participants"].iloc[i][j]["timeline"]["damageTakenPerMinDeltas"]["0-10"]
                except:
                    dtpm = 0
                    
                gold_per_minute_blue += gpm
                xp_per_minute_blue += xpm
                cs_per_minute_blue += cspm
                damage_taken_per_minute_blue += dtpm 
            else:
                champion_list_red.append(df["participants"].iloc[i][j]["championId"])
                try:
                    gpm = df["participants"].iloc[i][j]["timeline"]["goldPerMinDeltas"]["0-10"]
                except:
                    gpm = 0
                try:
                    xpm = df["participants"].iloc[i][j]["timeline"]["xpPerMinDeltas"]["0-10"]
                except:
                    xpm = 0
                try:
                    cspm = df["participants"].iloc[i][j]["timeline"]["creepsPerMinDeltas"]["0-10"]
                except:
                    cspm = 0
                try:
                    dtpm = df["participants"].iloc[i][j]["timeline"]["damageTakenPerMinDeltas"]["0-10"]
                except:
                    dtpm = 0
                    
                gold_per_minute_red += gpm
                xp_per_minute_red += xpm
                cs_per_minute_red += cspm
                damage_taken_per_minute_red += dtpm 
        
                
        global_result = [int(gameId), int(gameDuration), int(blueWins), int(firstblood_blue),\
                        int(firstTower_blue), int(firstDragon_blue)]
        
        champion_list = [champion_list_blue, champion_list_red]
        
        blue_data = [gold_per_minute_blue, xp_per_minute_blue,\
                     int(cs_per_minute_blue), int(damage_taken_per_minute_blue)]
             
        red_data = [int(gold_per_minute_red), int(xp_per_minute_red),\
                    int(cs_per_minute_red), int(damage_taken_per_minute_red)]
        
        all_data = np.array([global_result+ champion_list+ blue_data+red_data])
        
        columns_list = ["gameId", "gameDuration", "blueWins","First Blood",\
                        "First Tower", "First Dragon","champion_list_blue",\
                       "champion_list_red", "gpm_blue", "xp_pm_blue",\
                       "cs_pm_blue", "dmg_taken_blue", "gpm_red",\
                       "xp_pm_red", "cs_pm_red", "dmg_taken_red"]
        
        row_df = pd.DataFrame(all_data, columns = columns_list)
        
        df_storage = df_storage.append(row_df)
    return df_storage

In [8]:
prepared_df = pd.DataFrame()

In [9]:
def processing_df(df, splitter=1):
    prepared_df = pd.DataFrame()
    stepsize = len(df)/splitter
    for i in range(splitter):
        lower = int(i*stepsize)
        upper = int((i+1)*stepsize)
        prepared_df = prepared_df.append(preparing_dataframe(df,lower,upper))
        print(i)
    return prepared_df

In [10]:
processed_df = processing_df(match_df,splitter=13)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [11]:
working_df = processed_df.copy()

In [12]:
for i in range(len(working_df)):
    working_df["champion_list_blue"].iloc[i] = [eval(champs[champs["key"]==champion].iloc[0,2])\
                                                for champion in working_df["champion_list_blue"].iloc[i]]
    working_df["champion_list_red"].iloc[i] = [eval(champs[champs["key"]==champion].iloc[0,2])\
                                                for champion in working_df["champion_list_red"].iloc[i]]
    if i%2000==0:
        print(i)

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000


In [13]:
working_df.head()

,gameId,gameDuration,blueWins,First Blood,First Tower,First Dragon,champion_list_blue,champion_list_red,gpm_blue,xp_pm_blue,cs_pm_blue,dmg_taken_blue,gpm_red,xp_pm_red,cs_pm_red,dmg_taken_red
0,4247263043,1323,0,1,0,0,"[[Assassin, Mage], [Support, Mage], [Fighter, ...","[[Marksman, Support], [Marksman, Mage], [Mage]...",1237,1608.3,19,2370,1589,1837,22,2234
0,4247155821,1317,1,0,0,1,"[[Marksman], [Assassin, Fighter], [Support, Ma...","[[Marksman, Support], [Fighter, Marksman], [Ma...",1199.7,1674.1,22,1848,1339,1871,23,2267
0,4243963257,932,0,0,0,0,"[[Marksman, Support], [Mage, Fighter], [Suppor...","[[Tank, Support], [Marksman], [Assassin, Mage]...",1311.7,1617.8,22,2099,1790,1982,23,2181
0,4241678498,2098,0,0,0,1,"[[Tank, Fighter], [Mage, Support], [Marksman],...","[[Fighter, Tank], [Fighter], [Marksman], [Tank...",1318.4,1891.5,25,1963,1365,1876,25,1968
0,4241538868,2344,1,1,1,1,"[[Marksman], [Fighter, Tank], [Marksman], [Mag...","[[Fighter, Tank], [Fighter, Tank], [Mage, Assa...",1497.2,1863.2,25,1626,1308,1789,19,1895


In [15]:
working_df.shape

(22711, 16)

In [16]:
champs_list = []
for i in range(len(champs)):
    [champs_list.append(tag) for tag in eval(champs["tags"].loc[i]) if tag not in champs_list]

In [17]:
champs_list

['Fighter', 'Tank', 'Mage', 'Assassin', 'Support', 'Marksman']

# Creating a Column to count each type of champion + the team 

In [18]:
for item in champs_list:
    working_df[item+"_blue"]=0
    working_df[item+"_red"]=0

In [19]:
for i in range(len(working_df)):
    '''
    First process the blue champion types:
    '''
    for j in range(len(working_df["champion_list_blue"].iloc[i])):
        if "Fighter" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Fighter_blue"].iloc[i]+=1
            
        if "Tank" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Tank_blue"].iloc[i]+=1
            
        if "Mage" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Mage_blue"].iloc[i]+=1
            
        if "Assasin" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Assasin_blue"].iloc[i]+=1
            
        if "Support" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Support_blue"].iloc[i]+=1
            
        if "Marksman" in working_df["champion_list_blue"].iloc[i][j]:
            working_df["Marksman_blue"].iloc[i]+=1
            
    '''
    Then do the same with the red champions
    '''
    for j in range(len(working_df["champion_list_red"].iloc[i])):
        if "Fighter" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Fighter_red"].iloc[i]+=1
            
        if "Tank" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Tank_red"].iloc[i]+=1
            
        if "Mage" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Mage_red"].iloc[i]+=1
            
        if "Assasin" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Assasin_red"].iloc[i]+=1
            
        if "Support" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Support_red"].iloc[i]+=1
            
        if "Marksman" in working_df["champion_list_red"].iloc[i][j]:
            working_df["Marksman_red"].iloc[i]+=1


/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
working_df = working_df.drop(columns=["champion_list_blue","champion_list_red"])

In [27]:
working_df=working_df.reset_index(drop=True)

In [67]:
working_df["First Blood"]=working_df["First Blood"].astype(int)

In [69]:
working_df["First Tower"]=working_df["First Tower"].astype(int)

In [70]:
working_df["First Dragon"]=working_df["First Dragon"].astype(int)

In [72]:
working_df["blueWins"]=working_df["blueWins"].astype(int)

In [73]:
working_df.dtypes

gameId             object
gameDuration       object
blueWins            int64
First Blood         int64
First Tower         int64
First Dragon        int64
Fighter_blue        int64
Fighter_red         int64
Tank_blue           int64
Tank_red            int64
Mage_blue           int64
Mage_red            int64
Assassin_blue       int64
Assassin_red        int64
Support_blue        int64
Support_red         int64
Marksman_blue       int64
Marksman_red        int64
gpm_blue          float64
xp_pm_blue        float64
cs_pm_blue        float64
dmg_taken_blue    float64
gpm_red           float64
xp_pm_red         float64
cs_pm_red         float64
dmg_taken_red     float64
dtype: object

In [37]:
num_cols = ["gpm_blue", "xp_pm_blue", "cs_pm_blue", "dmg_taken_blue",\
            "gpm_red", "xp_pm_red", "cs_pm_red", "dmg_taken_red"]

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
std = StandardScaler()
scaled_fit = std.fit(working_df[num_cols])

In [41]:
scaled = pd.DataFrame(scaled_fit.transform(working_df[num_cols]), columns=num_cols)

working_df = working_df.drop(columns=num_cols, axis=1)
working_df = working_df.merge(scaled, left_index=True, right_index=True, how="inner")

In [146]:
train_cols=working_df.columns.to_list()

In [147]:
train_cols.remove("blueWins")

In [148]:
train_cols.remove("gameDuration")

In [149]:
"blueWins" in train_cols

False

In [150]:
target_col = "blueWins"

In [152]:
train_cols.remove("gameId")

In [74]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(working_df, test_size=.2, random_state=3)

train_X = train[train_cols].reset_index(drop=True)
train_y = train[target_col].reset_index(drop=True)
test_X = test[train_cols].reset_index(drop=True)
test_y = test[target_col].reset_index(drop=True)

In [75]:
train_X.dtypes

First Blood         int64
First Tower         int64
First Dragon        int64
Fighter_blue        int64
Fighter_red         int64
Tank_blue           int64
Tank_red            int64
Mage_blue           int64
Mage_red            int64
Assassin_blue       int64
Assassin_red        int64
Support_blue        int64
Support_red         int64
Marksman_blue       int64
Marksman_red        int64
gpm_blue          float64
xp_pm_blue        float64
cs_pm_blue        float64
dmg_taken_blue    float64
gpm_red           float64
xp_pm_red         float64
cs_pm_red         float64
dmg_taken_red     float64
dtype: object

In [76]:
def evaluate_model(train_X, train_y, test_X, test_y, model):
    ml_model = model
    ml_model.fit(train_X, train_y)
    
    predictions = ml_model.predict(test_X)
    acc = accuracy_score(test_y, predictions)
    
    print(f"Our model achieved an accuracy of {acc}")

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [78]:
evaluate_model(train_X, train_y, test_X, test_y, LogisticRegression())

Our model achieved an accuracy of 0.7541272287034999


In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [80]:
evaluate_model(train_X, train_y, test_X, test_y, DecisionTreeClassifier())

Our model achieved an accuracy of 0.6698217037200088


In [81]:
evaluate_model(train_X, train_y, test_X, test_y, RandomForestClassifier(n_estimators=100))

Our model achieved an accuracy of 0.74950473255558


In [82]:
evaluate_model(train_X, train_y, test_X, test_y, SGDClassifier())

Our model achieved an accuracy of 0.7517059211974466


In [86]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate

In [87]:
models = {"Logistic Regression": LogisticRegression(),
          "Decision Tree": DecisionTreeClassifier(random_state=42),
          "Random Forest": RandomForestClassifier(n_estimators=100, n_jobs=-1)}

In [88]:
def validate_model(model, X):
    validation_results = cross_validate(model,
                                        X,
                                        train_y.values.ravel(),
                                        cv=5,
                                        scoring="balanced_accuracy")
    acc = validation_results["test_score"].mean()
    print(f"Balanced Mean Accuracy Score: {acc}")

In [89]:
def evaluate_models(models, X):
    for key, value in models.items():
        print(f"Model: {key}")
        validate_model(value, X)
        print("--------------\n")

In [90]:
evaluate_models(models, train_X)

Model: Logistic Regression
Balanced Mean Accuracy Score: 0.7595226434627441
--------------

Model: Decision Tree
Balanced Mean Accuracy Score: 0.6675474410789921
--------------

Model: Random Forest
Balanced Mean Accuracy Score: 0.7530283009947656
--------------



In [91]:
selector_rf = RFECV(models["Random Forest"],
                    cv=5,
                    scoring="balanced_accuracy")

In [92]:
selector_rf.fit(train_X, train_y)

RFECV(cv=5,
      estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                       class_weight=None, criterion='gini',
                                       max_depth=None, max_features='auto',
                                       max_leaf_nodes=None, max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=-1,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False),
      min_features_to_select=1, n_jobs=None, scoring='balanced_accuracy',
      step=1, verbose=0)

In [93]:
selector_rf.get_support()

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [94]:
train_X.columns

Index(['First Blood', 'First Tower', 'First Dragon', 'Fighter_blue',
       'Fighter_red', 'Tank_blue', 'Tank_red', 'Mage_blue', 'Mage_red',
       'Assassin_blue', 'Assassin_red', 'Support_blue', 'Support_red',
       'Marksman_blue', 'Marksman_red', 'gpm_blue', 'xp_pm_blue', 'cs_pm_blue',
       'dmg_taken_blue', 'gpm_red', 'xp_pm_red', 'cs_pm_red', 'dmg_taken_red'],
      dtype='object')

In [95]:
train_X2 = train_X[train_X.columns[selector_rf.get_support()]]

In [96]:
evaluate_models(models, train_X2)

Model: Logistic Regression
Balanced Mean Accuracy Score: 0.7589173711811356
--------------

Model: Decision Tree
Balanced Mean Accuracy Score: 0.6642456280503032
--------------

Model: Random Forest
Balanced Mean Accuracy Score: 0.7532481107213082
--------------



In [97]:
models

{'Logistic Regression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'Decision Tree': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=42, splitter='best'),
 'Random Forest': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        ma

In [98]:
models["Random Forest"] = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [99]:
selector_rf2 = RFECV(models["Random Forest"],
                     cv=5,
                     scoring="balanced_accuracy")
selector_rf.fit(train_X, train_y)
train_X3 = train_X[train_X.columns[selector_rf.get_support()]]

In [100]:
evaluate_models(models, train_X3)

Model: Logistic Regression
Balanced Mean Accuracy Score: 0.7595226434627441
--------------

Model: Decision Tree
Balanced Mean Accuracy Score: 0.6652910652937626
--------------

Model: Random Forest
Balanced Mean Accuracy Score: 0.7552290348858917
--------------



In [101]:
from sklearn.model_selection import GridSearchCV

In [102]:
param_grid_rf = {"n_estimators": [500, 1000],
                 "criterion": ["gini", "entropy"],
                 "bootstrap": [True, False],
                 "max_depth": [3, 10, None],
                 "min_samples_leaf": [10, 30, 50, None]}

tune_rf = GridSearchCV(RandomForestClassifier(),
                       param_grid=param_grid_rf,
                       cv=3,
                       scoring="balanced_accuracy",
                       n_jobs=-1)

tune_rf.fit(train_X3, train_y)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [103]:
tune_rf.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 50,
 'n_estimators': 500}

In [105]:
validate_model(RandomForestClassifier(**tune_rf.best_params_), train_X3)

Balanced Mean Accuracy Score: 0.7574859561798869


In [112]:

train_X3 = train_X[train_X.columns[selector_rf.get_support()]]
rf_model = RandomForestClassifier(**tune_rf.best_params_)
rf_model.fit(train_X3, train_y)

test_X3 = test_X[test_X.columns[selector_rf.get_support()]]
rf_pred = rf_model.predict(test_X3)

In [113]:
evaluate_models(models, train_X3)

Model: Logistic Regression
Balanced Mean Accuracy Score: 0.7595226434627441
--------------

Model: Decision Tree
Balanced Mean Accuracy Score: 0.6652910652937626
--------------

Model: Random Forest
Balanced Mean Accuracy Score: 0.7568802596136828
--------------



In [ ]:
gameId             object
gameDuration       object
blueWins            int64
First Blood         int64
First Tower         int64
First Dragon        int64
Fighter_blue        int64
Fighter_red         int64
Tank_blue           int64
Tank_red            int64
Mage_blue           int64
Mage_red            int64
Assassin_blue       int64
Assassin_red        int64
Support_blue        int64
Support_red         int64
Marksman_blue       int64
Marksman_red        int64
gpm_blue          float64
xp_pm_blue        float64
cs_pm_blue        float64
dmg_taken_blue    float64
gpm_red           float64
xp_pm_red         float64
cs_pm_red         float64
dmg_taken_red     float64
dtype: object

In [137]:
num_cols = ["Fighter_blue", "Fighter_red", "Tank_blue","Tank_red",\
            "Mage_blue","Mage_red","Assassin_blue","Assassin_red",\
            "Support_blue","Support_red", "Marksman_blue", "Marksman_red",\
            "First Blood", "First Tower"]

In [138]:
std = StandardScaler()
scaled_fit = std.fit(working_df[num_cols])

In [139]:
scaled = pd.DataFrame(scaled_fit.transform(working_df[num_cols]), columns=num_cols)

working_df = working_df.drop(columns=num_cols, axis=1)
working_df = working_df.merge(scaled, left_index=True, right_index=True, how="inner")

In [140]:
train_cols=working_df.columns.to_list()

In [141]:
working_df.dtypes

gameId             object
gameDuration       object
blueWins            int64
First Dragon        int64
gpm_blue          float64
xp_pm_blue        float64
cs_pm_blue        float64
dmg_taken_blue    float64
gpm_red           float64
xp_pm_red         float64
cs_pm_red         float64
dmg_taken_red     float64
Fighter_blue      float64
Fighter_red       float64
Tank_blue         float64
Tank_red          float64
Mage_blue         float64
Mage_red          float64
Assassin_blue     float64
Assassin_red      float64
Support_blue      float64
Support_red       float64
Marksman_blue     float64
Marksman_red      float64
First Blood       float64
First Tower       float64
dtype: object

In [142]:
train_cols = num_cols.copy()

In [153]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(working_df, test_size=.2, random_state=3)

train_X = train[train_cols].reset_index(drop=True)
train_y = train[target_col].reset_index(drop=True)
test_X = test[train_cols].reset_index(drop=True)
test_y = test[target_col].reset_index(drop=True)

In [154]:
evaluate_model(train_X, train_y, test_X, test_y, LogisticRegression())

Our model achieved an accuracy of 0.7536869909751266


In [155]:
evaluate_models(models, train_X)

Model: Logistic Regression
Balanced Mean Accuracy Score: 0.7595226737687868
--------------

Model: Decision Tree
Balanced Mean Accuracy Score: 0.6672728683335798
--------------

Model: Random Forest
Balanced Mean Accuracy Score: 0.7551739688065965
--------------

